<a href="https://colab.research.google.com/github/rmhyps1/statistics/blob/main/TUGAS7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import math
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

try:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    HAVE_TUKEY = True
except Exception:
    HAVE_TUKEY = False

BASE_NAME = "Most Streamed Spotify Songs 2024"
OUT_PREFIX = "/mnt/data/anova_spotify"

def find_file_by_basename(base_name):
    search_dirs = [os.getcwd(), "/content", "/mnt/data"]
    candidates = []
    for d in search_dirs:
        if not os.path.isdir(d):
            continue
        patterns = [
            os.path.join(d, base_name + ".*"),
            os.path.join(d, base_name + "*.*"),
        ]
        for pat in patterns:
            candidates.extend(glob.glob(pat))
    if not candidates:
        return None
    csvs = [c for c in candidates if c.lower().endswith(".csv")]
    xlxs = [c for c in candidates if c.lower().endswith(".xlsx") or c.lower().endswith(".xls")]
    if csvs:
        return csvs[0]
    if xlxs:
        return xlxs[0]
    return candidates[0]

def read_table_auto(path):
    if path.lower().endswith(".csv") or path.lower().endswith(".txt"):
        encodings = ['utf-8', 'latin1', 'cp1252']
        last_exc = None
        for enc in encodings:
            try:
                df = pd.read_csv(path, encoding=enc)
                print(f"Read CSV with encoding: {enc} -> {path}")
                return df
            except Exception as e:
                last_exc = e
        raise last_exc
    else:
        try:
            df = pd.read_excel(path)
            print(f"Read Excel file -> {path}")
            return df
        except Exception as e:
            encodings = ['utf-8', 'latin1', 'cp1252']
            last_exc = e
            for enc in encodings:
                try:
                    df = pd.read_csv(path, encoding=enc)
                    print(f"Read CSV fallback with encoding: {enc} -> {path}")
                    return df
                except Exception as e2:
                    last_exc = e2
            raise last_exc

def detect_and_prepare_data(df):
    if df.shape[1] == 2:
        first_dtype = df.dtypes[0]
        if first_dtype == object or not np.issubdtype(first_dtype, np.number):
            group_col = df.columns[0]
            value_col = df.columns[1]
            groups = df[group_col].astype(str)
            values = pd.to_numeric(df[value_col], errors='coerce')
            data = {}
            for g in groups.unique():
                vals = values[groups == g].dropna().values.astype(float)
                if len(vals) > 0:
                    data[str(g)] = vals
            if len(data) > 0:
                print(f"Interpreted as LONG format: group='{group_col}', value='{value_col}'. Groups found: {len(data)}")
                return data
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_cols) == 0:
        coerced = []
        for c in df.columns:
            ser = pd.to_numeric(df[c], errors='coerce')
            if ser.notna().sum() > 0:
                df[c] = ser
                coerced.append(c)
        numeric_cols = coerced
    data = {}
    for c in numeric_cols:
        vals = df[c].dropna().values.astype(float)
        if len(vals) > 0:
            data[str(c)] = vals
    print(f"Interpreted as WIDE format with {len(data)} numeric treatment columns.")
    return data

def compute_anova_from_dict(data_dict):
    data = {k: np.asarray(v, dtype=float) for k, v in data_dict.items() if len(v) > 0}
    if len(data) == 0:
        raise ValueError("No groups with data provided.")
    k = len(data)
    T_list = []
    n_list = []
    y2 = 0.0
    for name, arr in data.items():
        n_i = arr.size
        T_i = arr.sum()
        T_list.append(T_i)
        n_list.append(n_i)
        y2 += (arr ** 2).sum()
    N = int(sum(n_list))
    T_total = float(sum(T_list))
    T2n = sum((T_list[i] ** 2) / n_list[i] for i in range(k))
    SSTr = T2n - (T_total ** 2) / N
    SST = y2 - (T_total ** 2) / N
    SSE = SST - SSTr
    dfTr = k - 1
    dfE = N - k
    dfT = N - 1
    MSTr = SSTr / dfTr if dfTr > 0 else float('nan')
    MSE = SSE / dfE if dfE > 0 else float('nan')
    F = MSTr / MSE if (not math.isnan(MSTr) and not math.isnan(MSE) and MSE != 0) else float('nan')
    p_value = stats.f.sf(F, dfTr, dfE) if (not math.isnan(F) and dfTr > 0 and dfE > 0) else float('nan')
    per_treatment = []
    for name, arr in data.items():
        per_treatment.append({
            'treatment': name,
            'n': arr.size,
            'sum': float(arr.sum()),
            'mean': float(arr.mean()) if arr.size > 0 else float('nan'),
            'var_sample': float(arr.var(ddof=1)) if arr.size > 1 else float('nan')
        })
    return {
        'k': k, 'N': N, 'T_total': T_total, 'y2': y2,
        'SSTr': SSTr, 'SST': SST, 'SSE': SSE,
        'dfTr': dfTr, 'dfE': dfE, 'dfT': dfT,
        'MSTr': MSTr, 'MSE': MSE, 'F': F, 'p_value': p_value,
        'per_treatment': pd.DataFrame(per_treatment),
        'raw_data': data
    }

def run_for_basename(basename, out_prefix=OUT_PREFIX, do_plot=True, do_tukey=True):
    found = find_file_by_basename(basename)
    if found is None:
        raise FileNotFoundError(f"No file found starting with '{basename}' in current directories.")
    print("Using file:", found)
    df = read_table_auto(found)
    print("DataFrame shape:", df.shape)
    display(df.head(6))
    data = detect_and_prepare_data(df)
    if len(data) == 0:
        raise ValueError("No usable numeric data detected in file.")
    res = compute_anova_from_dict(data)
    anova_df = pd.DataFrame({
        'Source': ['Treatment', 'Error', 'Total'],
        'SS': [res['SSTr'], res['SSE'], res['SST']],
        'df': [res['dfTr'], res['dfE'], res['dfT']],
        'MS': [res['MSTr'], res['MSE'], ""],
        'F': [res['F'], "", ""]
    })
    print("\n--- ANOVA Table (computed from pseudocode) ---")
    display(anova_df)
    print(f"\nk (treatments) = {res['k']}, N = {res['N']}")
    print(f"SSTr = {res['SSTr']:.6f}")
    print(f"SSE  = {res['SSE']:.6f}")
    print(f"SST  = {res['SST']:.6f}")
    print(f"dfTr = {res['dfTr']}, dfE = {res['dfE']}, dfT = {res['dfT']}")
    print(f"MSTr = {res['MSTr']:.6f}, MSE = {res['MSE']:.6f}")
    print(f"F = {res['F']:.6f}, p-value = {res['p_value']:.6e}")
    print("\n--- Per-treatment summary ---")
    display(res['per_treatment'])
    os.makedirs(os.path.dirname(out_prefix), exist_ok=True)
    summary_csv = f"{out_prefix}_summary.csv"
    anova_csv = f"{out_prefix}_anova_table.csv"
    pd.DataFrame([{
        'k': res['k'], 'N': res['N'],
        'SSTr': res['SSTr'], 'SSE': res['SSE'], 'SST': res['SST'],
        'dfTr': res['dfTr'], 'dfE': res['dfE'], 'dfT': res['dfT'],
        'MSTr': res['MSTr'], 'MSE': res['MSE'], 'F': res['F'], 'p_value': res['p_value']
    }]).to_csv(summary_csv, index=False)
    anova_df.to_csv(anova_csv, index=False)
    print(f"Saved summary to {summary_csv} and ANOVA table to {anova_csv}")
    if do_tukey:
        if not HAVE_TUKEY:
            print("statsmodels not installed: Tukey HSD skipped. To enable, run: pip install statsmodels")
        else:
            long_vals = []
            long_groups = []
            for name, arr in res['raw_data'].items():
                long_vals.append(arr)
                long_groups.extend([name] * arr.size)
            vals_concat = np.concatenate(long_vals)
            tuk = pairwise_tukeyhsd(endog=vals_concat, groups=np.array(long_groups), alpha=0.05)
            print("\n--- Tukey HSD results (pairwise comparisons) ---")
            print(tuk.summary())
            try:
                tuk_df = pd.DataFrame(data=tuk._results_table.data[1:], columns=tuk._results_table.data[0])
                tuk_path = f"{out_prefix}_tukey.csv"
                tuk_df.to_csv(tuk_path, index=False)
                print(f"Saved Tukey results to {tuk_path}")
            except Exception:
                pass
    return {
        'anova_table': anova_df,
        'summary': res['per_treatment'],
        'results': res,
        'source_file': found
    }

if __name__ == "__main__":
    result = run_for_basename(BASE_NAME, out_prefix=OUT_PREFIX, do_plot=True, do_tukey=True)
    print("\nDone. Results stored and displayed above.")


Using file: /content/Most Streamed Spotify Songs 2024.csv
Read CSV with encoding: latin1 -> /content/Most Streamed Spotify Songs 2024.csv
DataFrame shape: (4600, 29)


,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1
5,Lovin On Me,Lovin On Me,Jack Harlow,11/10/2023,USAT22311371,6,410.1,"670,665,438","105,892","175,421,034",...,"4,654",86.0,"17,167,254",152.0,"138,529,362","50,982","9,438,601","4,517,131",NaN,1


Interpreted as WIDE format with 6 numeric treatment columns.

--- ANOVA Table (computed from pseudocode) ---


,Source,SS,df,MS,F
0,Treatment,1.071827e+07,5,2143654.516215,1244.849751
1,Error,4.176412e+07,24253,1722.018673,
2,Total,5.248239e+07,24258,,



k (treatments) = 6, N = 24259
SSTr = 10718272.581075
SSE  = 41764118.869650
SST  = 52482391.450725
dfTr = 5, dfE = 24253, dfT = 24258
MSTr = 2143654.516215, MSE = 1722.018673
F = 1244.849751, p-value = 0.000000e+00

--- Per-treatment summary ---


,treatment,n,sum,mean,var_sample
0,Track Score,4600,192482.6,41.844043,1485.621878
1,Spotify Popularity,3796,241052.0,63.501581,262.000788
2,Apple Music Playlist Count,4039,220542.0,54.603120,5128.317187
3,Deezer Playlist Count,3679,118872.0,32.310954,2945.725467
4,Amazon Playlist Count,3545,89862.0,25.348942,675.471038
5,Explicit Track,4600,1651.0,0.358913,0.230145


Saved summary to /mnt/data/anova_spotify_summary.csv and ANOVA table to /mnt/data/anova_spotify_anova_table.csv

--- Tukey HSD results (pairwise comparisons) ---
                     Multiple Comparison of Means - Tukey HSD, FWER=0.05                     
          group1                     group2           meandiff p-adj  lower    upper   reject
---------------------------------------------------------------------------------------------
     Amazon Playlist Count Apple Music Playlist Count  29.2542   0.0  26.5324   31.976   True
     Amazon Playlist Count      Deezer Playlist Count    6.962   0.0   4.1787   9.7454   True
     Amazon Playlist Count             Explicit Track   -24.99   0.0 -27.6331 -22.3469   True
     Amazon Playlist Count         Spotify Popularity  38.1526   0.0  35.3904  40.9149   True
     Amazon Playlist Count                Track Score  16.4951   0.0   13.852  19.1382   True
Apple Music Playlist Count      Deezer Playlist Count -22.2922   0.0 -24.9874 -19.5969